In [1]:
!pip install requests beautifulsoup4

In [2]:
# live scraping
import requests as re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json

#  classification
from fastai.vision.all import *

#  recommendation
import re

In [110]:
class Classification_Recommendation():
    def __init__(self):
        self.names = []
        self.prices = []
        self.links = []
        self.imgs = []
    
    
    
    def access_webpage(self, url):
        driver = webdriver.Chrome('./chromedriver') 
        driver.get(url)
        html = driver.page_source 
#         print(html)
        soup = BeautifulSoup(html, 'html.parser')
        
        return soup
    
    
    
    def scrape_data(self, container, items, item_len):
        item_n = [] 
        item_link = []
        item_img = []
        item_price = []
        
        i = 0
        while i != item_len:
            for div in container:
                for item in items:
                    
                    img_len = len(items[2])
                    item_name = item[0].lower()
                    
                    data_fetched = div.findAll(item[1], class_ = item[2])

                    if item_name == 'name':
                        item_n.extend(data_fetched)
                    elif item_name == 'link':
                        item_link.extend(data_fetched)
                    elif item_name == 'image':
                        item_img.extend(data_fetched)
                    elif item_name == 'price':
                        item_price.extend(data_fetched)
                    
                    
                i += 1
                if i == item_len:
                    break
                    
                    
        return  item_n, item_link, item_img, item_price
    
    
    
    def link_data(self,ratings, links, stores):
        item_ratings = []

        for link in links:
            if stores.lower() == 'flipkart':
                item_link = f'https://www.flipkart.com{link["href"]}'
                soup = self.access_webpage(item_link)
            else: 
                soup = self.access_webpage(link['href'])
                    
            item_ratings.append(soup.findAll(ratings[0], class_ = ratings[1]))
                    
            
        return item_ratings
                
        
        
        
    def Filter(self, item_n, item_link, item_img, item_price, item_ratings):
        names = []
        prices = []
        links = []
        imgs = []
        ratings = []
        
        for name in item_n:
            names.append(name.get_text())
            
        for link in item_link:
            links.append(link['href'])
            
        for price in item_price:
            prices.append(price.get_text())
            
        for img in item_img:
            imgs.append(img['src'])
        
        for rating in item_ratings:
            ratings.append(price.get_text())
        
        return names, prices, links, imgs, ratings
            
            
        
            
    def scrape(self, directories):
        self.dataset = []
        
        for c in range(len(directories)):
            f = open(directories[c])
            parameters = json.load(f)
            
            for param in parameters:
                time.sleep(5)
                
                soup = self.access_webpage(param['url'])

                con = param['main_container']
                container = soup.findAll(con[0], class_ = con[1])
#                 print(container)

                item_n, item_link, item_img, item_price = self.scrape_data(container, param['fetch_data_item'], param['item_len'])
                
                item_ratings = self.link_data(param['Ratings'], item_link, param['Store'])
                names, prices, links, imgs, ratings = self.Filter(item_n, item_link, item_img, item_price, item_ratings)


                for i in range(param['item_len']):
                    data = {
                        'Name': names[i],
                        'Price': prices[i],
                        'Image': imgs[i],
                        'Link' : links[i],
                        'Ratings' : ratings[i],
                        'Category': param['Category'],
                        'Store' : param['Store'],
                        'Gender' : param['Gender']
                    }

                    self.dataset.append(data)

                print(f'{param["item_len"]}  {param["Category"]} items from {param["Store"]} has been scraped.')
            
        return self.dataset
    
    
    
    
    def CM_Models(self, gender, img):
        cm_path = "C:\\Users\\emielou\\Desktop\\scraping\\ClassificationModels\\"

        if gender.lower() == 'man':
            cmodel = load_learner(f'{cm_path}m_classification\\m_classification.pkl')
        else:
            cmodel = load_learner(f'{cm_path}w_classification\\w_classification.pkl')

        img_category = cmodel.predict(item = img)

        return img_category[0], gender
    
    
    
    
    def display_classified(self, img_category, gender):
        Items = []
        
        for data in self.dataset:
            Items.append(data)
        
        display_items = []
        for i in range(len(Items)):
            if Items[i]['Category'].lower() == img_category.lower() and Items[i]['Gender'].lower() == gender.lower():
                display_items.append(Items[i])
        
        
        return display_items
    
    
#     def data_pairs(self, dataset):
#         price cleaning
        
        
        
        
            



In [111]:
srp = Classification_Recommendation()

In [112]:
directory = ['C:/Users/emielou/Desktop/scraping/Queries/flipkart_queries.json']

In [ ]:
dataset = srp.scrape(directory)

C:\Users\emielou\AppData\Local\Temp\ipykernel_19588\2963413791.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [48]:
dataset

[{'Name': 'Men Solid Casual Jacket',
  'Price': '₹879',
  'Image': 'https://rukminim1.flixcart.com/image/612/612/xif0q/jacket/m/l/r/xl-1-no-fs2430-sovian-original-imagj2g5njt2ygtc.jpeg?q=70',
  'Link': '/sovian-full-sleeve-solid-men-jacket/p/itm45e03a5edff67?pid=JCKGJ2G5F5WPZVJG&lid=LSTJCKGJ2G5F5WPZVJG7UFZHO&marketplace=FLIPKART&q=jacket+for+man&store=clo%2Fqvw%2Fz0g%2Fjbm&srno=s_1_1&otracker=search&otracker1=search&iid=en_dAbhunEomXsj4Zv5u1o79UixZx1fBUOsRGPFyWnfIG5iZYx%2BHl23Umtu36OHeysqnUaSj1v9Nfa4%2FTBalu1wAA%3D%3D&ssid=v58gfuizeo0000001675684535734&qH=206ded8eaea6e204',
  'Category': 'Jacket',
  'Store': 'Flipkart',
  'Gender': 'Men'},
 {'Name': 'Men Solid Casual Jacket',
  'Price': '₹849',
  'Image': 'https://rukminim1.flixcart.com/image/612/612/xif0q/jacket/l/t/f/xxl-1-no-mens-jacket-7171-dolanto-original-imagkb3evqkzhxar.jpeg?q=70',
  'Link': '/dolanto-full-sleeve-solid-men-jacket/p/itmb17b43eb20b6f?pid=JCKGKB3EVWBQVH6Y&lid=LSTJCKGKB3EVWBQVH6YWT0F48&marketplace=FLIPKART&q=jacket

In [ ]:
img_path = 'C:\\Users\\emielou\\Desktop\\scraping\\ClassificationModels\\Trial_Image\man_short1.jpg'

img_category, gender = srp.CM_Models('men', img_path)
items = srp.display_classified(img_category, gender)

In [ ]:
price = []
for item in items:
    item_price = item['Price']
    if item['Store'].lower() == 'Flipkart'.lower():
        converted = float(item_price.strip().replace('₹',''))
    else:
        to_convert = re.findall(r'\d+\.\d+', item_price)
        converted = float(to_convert[0])
    
    price.append(converted)